## Imports and Utils

In [1]:
import sys; sys.path.append('..')
import matplotlib.pyplot as plt
import quimb.tensor as qtn
import cotengra as ctg
import quimb as qu
import numpy as np
import time, random

In [2]:

import pyzx as zx
import importlib
importlib.reload(zx)
zx.settings.drawing_backend = 'd3'
def show(g):
  return zx.draw_matplotlib(g,labels=True,h_edge_draw='box')
def show_d3(g):
  return zx.draw_d3(g)

In [3]:
def hbox_degree_fn(g, v):
    degree = 0
    for nbr in g.neighbors(v):
        for nbr2 in g.neighbors(nbr):
            if g.type(nbr2)==3 and nbr2 != v: 
                degree+=1
    return degree


def compute_stats(g):
    g1 = g.copy()
    from pyzx.hsimplify import to_hypergraph_form
    to_hypergraph_form(g1)
    count_hbox = 0
    count_z = 0
    count = 0
    one_arity = 0
    two_arity = 0
    three_arity = 0
    phase_hbox =0
    degree=0
    hbox_degree=0
    hbox_tensor_degree = 0
    for v in g1.vertices():
        arity = len(g1.neighbors(v))
        degree += arity
        hbox_degree += arity if g1.type(v)==3 else 0
        if g1.type(v)==3:
            phase_hbox += ((arity!=1) and (g1.phase(v) != 1))
            one_arity+= arity==1
            two_arity+= arity==2
            three_arity+= arity==3
            hbox_tensor_degree += hbox_degree_fn(g1, v)
        count_hbox+= g1.type(v)==3
        count_z+= g1.type(v)==1
        count+=1
    print(f'H_box/Total = {count_hbox/count}')
    print(f'Z/Total = {count_z/count}')
    print(f'Multiarity Hbox/Total = {(two_arity+three_arity)/count}')
    print(f'Multiarity phase Hbox/Total = {phase_hbox/count_hbox}')
    print(f'total num of Z = {count_z}')
    print(f'total num of Hbox = {count_hbox}')
    print(f'One arity Hboxes = {one_arity}')
    print(f'Two arity Hboxes = {two_arity}')
    print(f'Three arity Hboxes = {three_arity}')
    print(f'Average degree = {degree/count}')
    print(f'Average Hbox degree = {hbox_degree/count_hbox}')
    print(f'Average Hbox tensor degree = {hbox_tensor_degree/count_hbox}')

In [4]:
def fourier_transform_2(g):
    from pyzx.utils import EdgeType, VertexType
    del_e = []
    del_v = []
    add_e = []
    types = g.types()
    phases = g.phases()
    es = list(g.edges())
    vs = list(g.vertices())

    for v in vs:
        arity = len(g.neighbors(v))
        if types[v] == VertexType.H_BOX and arity == 2 and phases[v] != 1:
            phase = phases[v]
            v1,v2 = g.neighbors(v)
            c = g.add_vertex(VertexType.Z)
            vh1 = g.add_vertex(VertexType.H_BOX)
            vh2 = g.add_vertex(VertexType.H_BOX)
            ch1 = g.add_vertex(VertexType.H_BOX)
            c2 = g.add_vertex(VertexType.Z)
            ch2 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh1))
            add_e.append(g.edge(v2, vh2))
            add_e.append(g.edge(vh1, c))
            add_e.append(g.edge(vh2, c))
            add_e.append(g.edge(c, ch1))
            add_e.append(g.edge(ch1, c2))
            add_e.append(g.edge(c2, ch2))
            g.set_phase(ch2, phase*(-1/2))
            g.set_phase(v1, phases[v1]+phase*1/2)
            g.set_phase(v2, phases[v2]+phase*1/2)
            g.scalar.add_power(-2)
            # g.scalar.add_float(1/2)
            del_v.append(v)
            del_e.append(g.edge(v,v1))
            del_e.append(g.edge(v,v2))


    g.add_edges(add_e)
    g.remove_edges(del_e)
    for v in del_v:
        g.remove_vertex(v)

def fourier_transform_3(g):
    from pyzx.utils import EdgeType, VertexType
    del_e = []
    del_v = []
    add_e = []
    types = g.types()
    phases = g.phases()
    es = list(g.edges())
    vs = list(g.vertices())

    for v in vs:
        arity = len(g.neighbors(v))
        if types[v] == VertexType.H_BOX and arity == 3:
            phase = phases[v]
            v1,v2,v3 = g.neighbors(v)
            c = g.add_vertex(VertexType.Z)
            vh1 = g.add_vertex(VertexType.H_BOX)
            vh2 = g.add_vertex(VertexType.H_BOX)
            vh3 = g.add_vertex(VertexType.H_BOX)
            ch1 = g.add_vertex(VertexType.H_BOX)
            c2 = g.add_vertex(VertexType.Z)
            ch2 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh1))
            add_e.append(g.edge(v2, vh2))
            add_e.append(g.edge(v3, vh3))
            add_e.append(g.edge(vh1, c))
            add_e.append(g.edge(vh2, c))
            add_e.append(g.edge(vh3, c))
            add_e.append(g.edge(c, ch1))
            add_e.append(g.edge(ch1, c2))
            add_e.append(g.edge(c2, ch2))
            vh12 = g.add_vertex(VertexType.H_BOX)
            vh23 = g.add_vertex(VertexType.H_BOX)
            vh31 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh12))
            add_e.append(g.edge(vh12, v2))
            add_e.append(g.edge(v2, vh23))
            add_e.append(g.edge(vh23, v3))
            add_e.append(g.edge(v3, vh31))
            add_e.append(g.edge(vh31, v1))
            g.set_phase(ch2, phase*(1/4) )
            g.set_phase(vh12, phase*(1/2))
            g.set_phase(vh23, phase*(1/2) )
            g.set_phase(vh31, phase*(1/2) )
            g.set_phase(v1, phases[v1]-phase*1/4)
            g.set_phase(v2, phases[v2]-phase*1/4)
            g.set_phase(v3, phases[v3]-phase*1/4)
            g.scalar.add_power(-2)
            # g.scalar.add_float(1/2)
            del_v.append(v)
            del_e.append(g.edge(v,v1))
            del_e.append(g.edge(v,v2))
            del_e.append(g.edge(v,v3))


    g.add_edges(add_e)
    g.remove_edges(del_e)
    for v in del_v:
        g.remove_vertex(v)

def val(g1):
    return abs(zx.to_quimb_tensor(g1).contract(output_inds={}))

In [5]:
def compute_contraction_costs(tn, progbar=True):
    print(f'#Tensors={tn.num_tensors}, #Indices={tn.num_indices}')
    print("-"*10+"-> Applying quimb's full_simplify ...")
    tn.full_simplify_('ADCSRL',output_inds=(),  progbar=True)
    print(f'#Tensors={tn.num_tensors}, #Indices={tn.num_indices}')
    if tn.num_tensors==1:
        return
    ctg.hyper._HYPER_SEARCH_SPACE['kahypar']['imbalance']['max'] = 0.4
    opt = ctg.HyperOptimizer(
        methods=['kahypar'],
        max_repeats=128,
        progbar=progbar,
        minimize='flops'
    )
    print("-"*10+"-> Finding contraction tree...")
    info = tn.contract(all, optimize=opt,output_inds=(), get='path-info')
    tree = opt.get_tree()
    print(f' Contraction cost = {tree.contraction_cost():e} \n Memory cost = {tree.contraction_width():e}')
    return opt

## Main

### Quimb Tensor Contraction

In [6]:
file = 'circuits/depth_7.qsim'

In [52]:
circ2 = qtn.Circuit.from_qasm_file(file, gate_opts={})
psi_f = qtn.MPS_computational_state('0' * (circ2.N))
tn2 = circ2.psi  & psi_f
output_inds = []
# print(f'{tn2.contraction_cost(optimize="hyper-greedy"):e}')
opt = compute_contraction_costs(tn2, progbar=True)

#Tensors=1295, #Indices=1447
-----------> Applying quimb's full_simplify ...


L 104, 203: : 12it [00:03,  3.59it/s]   


#Tensors=104, #Indices=203
-----------> Finding contraction tree...


log2[SIZE]: 17.00 log10[FLOPs]: 7.73: 100%|██████████| 128/128 [00:18<00:00,  6.77it/s]

 Contraction cost = 2.712564e+07 
 Memory cost = 1.700000e+01


### Pyzx tensor contraction

In [7]:
file = 'circuits/depth_7.qsim'

In [8]:

circ1 = zx.Circuit.from_qsim_file(file)
g1 = circ1.to_graph(zh=True)
qubits = circ1.qubits
g1.apply_state("0"*qubits)
g1.apply_effect("0"*qubits)

In [55]:
g1.num_vertices()

5521

In [28]:
val(g1)

2.0816066857361734e-08

In [9]:
compute_stats(g1)

H_box/Total = 0.5818057870019694
Z/Total = 0.4181942129980306
Multiarity Hbox/Total = 0.35706711104378125
Multiarity phase Hbox/Total = 0.0
total num of Z = 5521
total num of Hbox = 7681
One arity Hboxes = 2967
Two arity Hboxes = 4714
Three arity Hboxes = 0
Average degree = 2.131040751401303
Average Hbox degree = 1.6137221715922405
Average Hbox tensor degree = 2.417133185783101


In [32]:
lcomp_matches, pivot_matches = zx.heuristics.simplify.generate_matches(g1, boundaries=True, gadgets=True, max_v=g1.num_vertices(), cap=-5)
len(lcomp_matches), len(pivot_matches)

(2776, 387)

In [34]:
a[-1]

(-4.0, (4635, [3944, 4636]), 3)

In [33]:
a = lcomp_matches
a.sort(key= lambda m: m[2],reverse=False)
a[:5]

[(-1.0, (62, [61, 63]), 1),
 (-1.0, (78, [77, 79]), 1),
 (-1.0, (93, [92, 94]), 1),
 (-1.0, (106, [105, 107]), 1),
 (-1.0, (111, [110, 112]), 1)]

In [10]:
from pyzx.simplify import greedy_simp, greedy_simp_neighbors, simulated_annealing_simp
greedy_simp(g1, boundaries=True, gadgets=True, max_v=g1.num_vertices(), cap=-1, quiet=False)
# zx.full_reduce(g1, quiet=False)
# simulated_annealing_simp(g1, iterations=1500, quiet=False)

spider_simp: 812. 510. 171. 26.  4 iterations
id_simp: 472.  1 iterations
spider_simp: 163. 154. 2.  3 iterations
id_simp: 173.  1 iterations
spider_simp: 163. 10.  2 iterations


2

In [57]:
compute_stats(g1)

H_box/Total = 0.6794126199476592
Z/Total = 0.3205873800523408
Multiarity Hbox/Total = 0.4216341959872056
Multiarity phase Hbox/Total = 0.0
total num of Z = 2205
total num of Hbox = 4673
One arity Hboxes = 1773
Two arity Hboxes = 2900
Three arity Hboxes = 0
Average degree = 2.2020936318697295
Average Hbox degree = 1.6205863471003639
Average Hbox tensor degree = 4.5786432698480635


In [13]:
compute_stats(g1)

H_box/Total = 0.7047493403693932
Z/Total = 0.29525065963060687
Multiarity Hbox/Total = 0.495778364116095
Multiarity phase Hbox/Total = 0.0
total num of Z = 1119
total num of Hbox = 2671
One arity Hboxes = 792
Two arity Hboxes = 1879
Three arity Hboxes = 0
Average degree = 2.401055408970976
Average Hbox degree = 1.703481842006739
Average Hbox tensor degree = 7.241482590789967


In [11]:
from pyzx.hsimplify import zh_simp, new_simp
new_simp(g1)

copy_simp: 39.  1 iterations
spider_simp: 40. 2.  2 iterations
hpivot: 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

4

In [59]:
compute_stats(g1)

H_box/Total = 0.7509937535491198
Z/Total = 0.24900624645088018
Multiarity Hbox/Total = 0.5309483248154457
Multiarity phase Hbox/Total = 0.3051039697542533
total num of Z = 877
total num of Hbox = 2645
One arity Hboxes = 775
Two arity Hboxes = 1686
Three arity Hboxes = 184
Average degree = 2.6683702441794437
Average Hbox degree = 1.7765595463137995
Average Hbox tensor degree = 9.289981096408317


In [39]:
val(g1)

2.0816066857347768e-08

In [12]:
fourier_transform_3(g1)
# new_simp(g1)

In [106]:
val(g1)

2.5901585500145456e-09

In [13]:

zx.hsimplify.par_hbox_simp(g1, quiet=False)

par_hbox_simp: 475.  1 iterations


1

In [108]:
val(g1)

2.5901585500147458e-09

In [19]:
compute_stats(g1)

H_box/Total = 0.719720024345709
Z/Total = 0.2802799756542909
Multiarity Hbox/Total = 0.49056603773584906
Multiarity phase Hbox/Total = 0.061733615221987316
total num of Z = 921
total num of Hbox = 2365
One arity Hboxes = 753
Two arity Hboxes = 1612
Three arity Hboxes = 0
Average degree = 2.420572124163116
Average Hbox degree = 1.6816067653276956
Average Hbox tensor degree = 7.016490486257928


In [14]:
fourier_transform_2(g1)
# new_simp(g1)

In [111]:
val(g1)

2.590158550014806e-09

In [33]:
new_simp(g1)

id_simp: 435.  1 iterations


1

In [112]:
val(g1)

2.590158550014806e-09

In [15]:
compute_stats(g1)

H_box/Total = 0.7034576733094873
Z/Total = 0.29654232669051267
Multiarity Hbox/Total = 0.4804973599046159
Multiarity phase Hbox/Total = 0.0
total num of Z = 1741
total num of Hbox = 4130
One arity Hboxes = 1309
Two arity Hboxes = 2821
Three arity Hboxes = 0
Average degree = 2.3679100664282062
Average Hbox degree = 1.6830508474576271
Average Hbox tensor degree = 6.564648910411623


In [114]:
show_d3(g1)

In [138]:
tn1 = zx.to_quimb_tensor(g1)
abs(tn1.contract(output_inds={}))

2.590158550014806e-09

In [146]:
tn1 = zx.to_quimb_tensor(g1)
tn1.compress_simplify_(output_inds={}, progbar=True)

S 1011, 458: : 15it [00:00, 22.65it/s] 
L 1106, 1659: : 6it [00:02,  2.75it/s]  
S 287, 208: : 10it [00:00, 18.17it/s]  
L 464, 696: : 6it [00:00,  6.83it/s]  
S 266, 206: : 10it [00:00, 32.40it/s] 
L 462, 693: : 6it [00:01,  5.47it/s] 
S 262, 206: : 10it [00:00, 31.97it/s]
L 462, 693: : 6it [00:00,  6.89it/s]  
S 261, 206: : 10it [00:00, 31.79it/s] 


<TensorNetwork(tensors=261, indices=206)>

In [147]:
opt = compute_contraction_costs(tn1)

#Tensors=261, #Indices=206
-----------> Applying quimb's full_simplify ...
#Tensors=261, #Indices=206
-----------> Finding contraction tree...


log2[SIZE]: 12.00 log10[FLOPs]: 5.78: 100%|██████████| 128/128 [00:33<00:00,  3.80it/s]

 Contraction cost = 3.024280e+05 
 Memory cost = 1.200000e+01


In [16]:
tn1 = zx.to_quimb_tensor(g1)
tn1.full_simplify_('ADCRSL',output_inds={}, progbar=True)

L 627, 471: : 30it [05:53, 11.77s/it]   


<TensorNetwork(tensors=627, indices=471)>

In [66]:
tn1.exponent

0.0

In [23]:
abs(tn1.contract(output_inds={}))

2.5901585500139417e-09

In [17]:
opt = compute_contraction_costs(tn1)

#Tensors=627, #Indices=471
-----------> Applying quimb's full_simplify ...


L 626, 470: : 12it [00:08,  1.38it/s]


#Tensors=626, #Indices=470
-----------> Finding contraction tree...


log2[SIZE]: 18.00 log10[FLOPs]: 7.84: 100%|██████████| 128/128 [01:42<00:00,  1.25it/s]

 Contraction cost = 3.430654e+07 
 Memory cost = 1.800000e+01


In [77]:
tn1 = zx.to_quimb_tensor(g1)
opt = compute_contraction_costs(tn1)

#Tensors=2725, #Indices=1223
-----------> Applying quimb's full_simplify ...


L 295, 177: : 36it [01:45,  2.92s/it]


#Tensors=295, #Indices=177
-----------> Finding contraction tree...


log2[SIZE]: 11.00 log10[FLOPs]: 5.51: 100%|██████████| 256/256 [01:25<00:00,  2.98it/s]

 Contraction cost = 1.606720e+05 
 Memory cost = 1.100000e+01


In [119]:
abs(tn1.contract(output_inds={}))

2.590158550015275e-09

In [118]:
tn1.contract(output_inds={})

(-2.3310527639639207e-09-1.129209602258765e-09j)

In [114]:
%%timeit
tn1.contract(all, optimize=opt.path, backend='jax', output_inds={})

3.82 ms ± 388 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
%%time
b = tn1.contract(all, optimize=opt.path, backend='jax', output_inds={})

CPU times: user 5.17 s, sys: 437 ms, total: 5.61 s
Wall time: 6.07 s


In [117]:
b

(nan+nanj)

In [116]:
abs(b)

nan

In [54]:
b = tn1.contract()
abs(b)

0.0

In [58]:
tn1.contract(output_inds={})

(-0+0j)

In [23]:
abs(b)

9.489816803518586e-27

In [52]:
c = zx.to_quimb_tensor(g1).contract(output_inds={})

In [53]:
abs(c)

2.590158550015271e-09